In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Thu Oct  5 03:24:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [51]:
from huggingface_hub import notebook_login

notebook_login()


## LLaMA2 7B Chat


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [52]:

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                            #  device_map='auto',
                                            #  torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                             load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
# from transformers import AutoTokenizer
# import transformers
# import torch

# model = "codellama/CodeLlama-7b-hf"

# tokenizer = AutoTokenizer.from_pretrained(model)
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

# sequences = pipeline(
#     'import socket\n\ndef ping_exponential_backoff(host: str):',
#     do_sample=True,
#     top_k=10,
#     temperature=0.1,
#     top_p=0.95,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     max_length=200,
# )
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")


In [ ]:
!nvidia-smi

Thu Oct  5 03:28:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |   4887MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
you are the sicientific information provider you provide detailed explaination for any scientific text passed. if you do not know about it
say you will get back once you have knowledge consider the below sample examples to genrate the result

text =


"""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        return wrapped_text +'\n\n'


In [ ]:
instruction = "What is the temperature in Melbourne?"

get_prompt(instruction)

'[INST]<<SYS>>\nyou are the sicientific information provider you provide detailed explaination for any scientific text passed. if you do not know about it\nsay you will get back once you have knowledge consider the below sample examples to genrate the result\n\ntext =\n\n\n\n<</SYS>>\n\nWhat is the temperature in Melbourne?[/INST]'

In [ ]:
instruction = "Summarize the following text for me {text}"

system_prompt = "You are an expert and summarization and reducing the number of words used"

get_prompt(instruction, system_prompt)

'[INST]<<SYS>>\nYou are an expert and summarization and reducing the number of words used\n<</SYS>>\n\nSummarize the following text for me {text}[/INST]'

## LangChain basics

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


In [ ]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.2})

In [ ]:
system_prompt1 = """ you are a table data"""
instruction1 = """firstly understand the text in detailed manner. return the key points from the description \n\n{text}"""
template1 = get_prompt(instruction1, system_prompt1)
print(template1)

prompt1 = PromptTemplate(template=template1, input_variables=["text"])
llm_chain_description = LLMChain(prompt=prompt1, llm=llm,output_key="description")

[INST]<<SYS>>
 you are a table data
<</SYS>>

firstly understand the text in detailed manner. return the key points from the description 

{text}[/INST]


In [ ]:
system_prompt1 = """ you are the sicientific information provider you provide detailed explaination for any scientific text passed. if you do not know about it
say you will get back once you have knowledge"""
instruction1 = """firstly understand the text in detailed manner. return the key points from the description \n\n{text}"""
template1 = get_prompt(instruction1, system_prompt1)
print(template1)

prompt1 = PromptTemplate(template=template1, input_variables=["text"])
llm_chain_description = LLMChain(prompt=prompt1, llm=llm,output_key="description")

[INST]<<SYS>>
 you are the sicientific information provider you provide detailed explaination for any scientific text passed. if you do not know about it
say you will get back once you have knowledge
<</SYS>>

firstly understand the text in detailed manner. return the key points from the description 

{text}[/INST]


In [ ]:
text = """CAN messages and its working in details
"""

description = llm_chain_description.run(text)

In [ ]:
description

"  Sure, I'd be happy to help! Please provide the scientific text you'd like me to explain, and I'll do my best to provide a detailed explanation. If I don't know something, I'll let you know and promise to get back to you once I have the knowledge you're looking for.\nFor the text you provided on CAN messages and their working, here are the key points:\n1. CAN (Controller Area Network) is a serial communication protocol used in vehicles to transmit messages between electronic control units (ECUs).\n2. CAN is a multi-drop network, meaning that any ECU on the network can transmit messages to any other ECU.\n3. CAN messages are structured in a specific format, consisting of a header, a checksum, and a data section. The header contains information about the message, such as the message type and the ECU it's coming from. The checksum is calculated using a checksum algorithm and is used to detect errors in the message. The data section contains the actual data being transmitted.\n4. There a

In [ ]:
import langchain

import time



def generate_text_with_feedback(llm, text,feedback_1):

  system_prompt1 = """ you are the sicientific information provider you provide detailed explaination for any scientific text passed. if you do not know about it
  say you will get back once you have knowledge"""
  instruction1 = """firstly understand the text in detailed manner. return the key points from the description \n if there are any speciafication in feedback consider them and genrate the result {feedback_1} \
  n\n{text} \n"""
  template1 = get_prompt(instruction1, system_prompt1)
  print(template1)

  prompt1 = PromptTemplate(template=template1, input_variables=["text","feedback_1"])
  llm_chain_description = LLMChain(prompt=prompt1, llm=llm,output_key="description")
  description = llm_chain_description.run({"text":text, "feedback_1":feedback_1})
  print(description)

  # Prompt the user for feedback.

  feedback1 = input("\n***Is this text satisfactory? (y/n): ")

  # If the user is not satisfied, generate a new text.

  if feedback1 != "y":
    feedback_1 = input("\n enter your comments here:")
    description = generate_text_with_feedback(llm, text,feedback_1)


  return description

text = """21L: Suitable for bachelors & small families
Convection Microwave Oven: Can be used for baking along with grilling, reheating, defrosting and cooking
1 year standard warranty on Product, 10 years warranty on ceramic enamel cavity
Control : Touch Key Pad (Membrane) is sensitive to touch and easy to clean
No.of Programs : Keep Warm, Power Defrost, Clock, Indian Recipe, Cafeteria, Tikka Series, Healthy Cook, Kid's Nutrition, Veggie, Microwave, Grill, Convection, Combi (Microwave + Grill), Combi (Microwave + Convection), Fast Preheat, 30 Sec Plus
Child Safety Lock: Ensures complete safety especially for homes with young children
"""

# text = """During LV Battery charging if the increase in SOC% <=1% for a duration of 15min,
# OBC shall indicate charge Timeout fault message to VCU."""
generated_text = generate_text_with_feedback(llm,text,feedback_1='none')



# Print the generated text.

print(generated_text)



[INST]<<SYS>>
 you are the sicientific information provider you provide detailed explaination for any scientific text passed. if you do not know about it
  say you will get back once you have knowledge
<</SYS>>

firstly understand the text in detailed manner. return the key points from the description 
 if there are any speciafication in feedback consider them and genrate the result {feedback_1}   n
{text} 
[/INST]
  Sure, I'd be happy to help you understand the text and provide a detailed explanation!
The text describes a scenario where the State of Charge (SOC) of a Lithium-Ion (Li-ion) battery is being charged, and there are certain specifications that need to be met during this process. Here are the key points from the description:
1. During LV battery charging, if the increase in SOC% is <=1% for a duration of 15 minutes, the On-Board Charger (OBC) shall indicate a "Charge Timeout fault message" to the Vehicle Control Unit (VCU).
Let me break down what this means:
* "LV battery ch

In [ ]:
print(description)

In [ ]:
system_prompt2 = """ you are a test case generator your job is to genreate all possible test cases for the given details.
if you are not able to understand returen a message saying i need more information"""
instruction2 = """for the given description generate the test case titles and send in json format \n\n {description}"""
template2 = get_prompt(instruction2, system_prompt2)
print(template2)

prompt2 = PromptTemplate(template=template2, input_variables=["description"])
llm_chain_test_case= LLMChain(prompt=prompt2, llm=llm, output_key="test_case")

res = llm_chain_test_case.run(description)

[INST]<<SYS>>
 you are a test case generator your job is to genreate all possible test cases for the given details.
if you are not able to understand returen a message saying i need more information
<</SYS>>

for the given description generate the test case titles and send in json format 

 {description}[/INST]


KeyboardInterrupt: ignored

In [ ]:
print(res)

In [ ]:
from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[llm_chain_description, llm_chain_test_case],
                                      input_variables=['text'],
                                      output_variables=['description','test_case'],
                                      verbose=True)

In [ ]:
text = """21L: Suitable for bachelors & small families
Convection Microwave Oven: Can be used for baking along with grilling, reheating, defrosting and cooking
1 year standard warranty on Product, 10 years warranty on ceramic enamel cavity
Control : Touch Key Pad (Membrane) is sensitive to touch and easy to clean
No.of Programs : Keep Warm, Power Defrost, Clock, Indian Recipe, Cafeteria, Tikka Series, Healthy Cook, Kid's Nutrition, Veggie, Microwave, Grill, Convection, Combi (Microwave + Grill), Combi (Microwave + Convection), Fast Preheat, 30 Sec Plus
Child Safety Lock: Ensures complete safety especially for homes with young children
Special Features : Various Cooking Mode, Convection Temperature (40~200 ℃), Preheat, Keep Warm, Auto Programs, Auto Cook, Sound on/off, Ceramic Enamel Cavity
Included in the Box : 1 Unit Convection Microwave Oven, 1 Unit Glass Turntable, 1 Wire Rack, 1 Quick Guide Label
"""

result = overall_chain(text)

In [ ]:
print(result['test_case'])

In [ ]:
import pandas as pd
import re
def strip(text):
  text = str(text)
  print(text)
  test_case_pattern = r"Test Case \d+: ([^\n]*)"
  test_condition_pattern = r"Test Condition: ([^\n]*)"
  expected_result_pattern = r"Expected Result: ([^\n]*)"
  matches_tc = re.findall(test_case_pattern,text,re.DOTALL)
  match_tcon = re.findall(test_condition_pattern,text,re.DOTALL)
  match_er = re.findall(expected_result_pattern,text,re.DOTALL)
  print(len(matches_tc),len(match_tcon),len(match_er))
  data = dict()
  data['test_case'] = matches_tc
  data['test condition'] =match_tcon
  data['expected result'] = match_er
  data1=list(data)
  df =pd.DataFrame.from_records(data1)
  return data,df


In [ ]:
text = """During LV Battery charging if the increase in SOC% <=1% for a duration of 15min,
 OBC shall indicate charge Timeout fault message to VCU.
"""
#240-Volt Outlet (Dryer Plug)"
output = llm_chain.run(text)

# data,df = strip(output)
# print(data)
# # print(df)

In [ ]:
text = """Duracell 38W Fast Car Charger Adapter with Dual Output.
Quick Charge, Type C PD 20W & Qualcomm Certified 3.0
Compatible for iPhone, All Smartphones, Tablets & More (Copper & Black)"""
#240-Volt Outlet (Dryer Plug)"
output = llm_chain.run(text)


data,df = strip(output)
print(data)
print(df)

### Summarization

In [ ]:
instruction = "Summarize the following article for me {text}"
system_prompt = "You are an expert and summarization and expressing key ideas succintly"

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
def count_words(input_string):
    words = input_string.split(" ")
    return len(words)

text = '''Twitter (now X) CEO Linda Yaccarino claims usage at ‘all time high’ in memo to staff
Twitter’s (now X’s) newly established CEO Linda Yaccarino touts the company’s success and X’s future plans in a company-wide memo obtained by CNBC. The exec once again claims, without sharing any specific metrics, that the service’s usage is at an “all time high,” and hints at what’s to come in terms of new product experiences for the newly rebranded platform.

The service formerly known as Twitter has been working to become more than just a social network and more of an “everything app,” as owner Elon Musk dubbed it.

As the Telsa and Space X exec explained in October 2022, telegraphing Twitter’s eventual rebranding, buying Twitter was meant to be “an accelerant to creating X, the everything app.”


His grand plan has been to create an app that allows creators to monetize their content, then later moves into payments services and even banking, Musk remarked during a Twitter Spaces livestream with advertisers in November. At the time, he even mentioned the possibility of establishing money market accounts on Twitter that would pay a high-interest rate to attract consumers to X.

Those possible product concepts were again referenced in Yaccarino’s new missive, when she writes, “Our usage is at an all time high and we’ll continue to delight our entire community with new experiences in audio, video, messaging, payments, banking – creating a global marketplace for ideas, goods, services, and opportunities.”

Twitter, now X, has already implemented some of Musk’s ideas around videos and creator monetization. In May, the company began allowing subscribers to upload two-hour videos to its service, which advertiser Apple then leveraged when it released the entire first episode of its hit Apple TV+ show “Silo” on the platform. Fired Fox News host Tucker Carlson had been posting lengthy videos to Twitter as well, until ordered to stop by the network.

In addition, earlier this month, Twitter began sharing ad revenue with verified creators.

However, all is not well at Twitter X, whose traffic — at least by third-party measurements — has been dropping. Data from web analytics firm Similarweb indicated Twitter’s web traffic declined 5% for the first two days its latest rival, Instagram Threads, became generally available, compared with the week prior. Plus, Similarweb said Twitter’s web traffic was down 11% compared with the same days in 2022. Additionally, Cloudflare CEO Matthew Prince earlier this month tweeted a graph of traffic to the Twitter.com domain that showed “Twitter traffic tanking,” he said.


Yaccarino subtly pushed back at those reports at the time, claiming that Twitter had its largest usage day since February in early July. She did not share any specific metrics or data. At the same time, however, the company was quietly blocking links to Threads.net in Twitter searches, suggesting it was concerned about the new competition.

Today, Yaccarino repeats her vague claims around X’s high usage in her company-wide memo even as analysts at Forrester are predicting X will either shut down or be acquired within the next 12 months and numerous critics concur that the X rebrand is destined to fail.

Yaccarino’s memo, otherwise, was mostly a lot of cheerleading, applauding X’s team for their work and touting X’s ability to “impress the world all over again,” as Twitter once did.

The full memo, courtesy of CBNC, is below:

Hi team,

What a momentous weekend. As I said yesterday, it’s extremely rare, whether it’s in life or in business, that you have the opportunity to make another big impression. That’s what we’re experiencing together, in real time. Take a moment to put it all into perspective.

17 years ago, Twitter made a lasting imprint on the world. The platform changed the speed at which people accessed information. It created a new dynamic for how people communicated, debated, and responded to things happening in the world. Twitter introduced a new way for people, public figures, and brands to build long lasting relationships. In one way or another, everyone here is a driving force in that change. But equally all our users and partners constantly challenged us to dream bigger, to innovate faster, and to fulfill our great potential.

With X we will go even further to transform the global town square — and impress the world all over again.

Our company uniquely has the drive to make this possible. Many companies say they want to move fast — but we enjoy moving at the speed of light, and when we do, that’s X. At our core, we have an inventor mindset — constantly learning, testing out new approaches, changing to get it right and ultimately succeeding.

With X, we serve our entire community of users and customers by working tirelessly to preserve free expression and choice, create limitless interactivity, and create a marketplace that enables the economic success of all its participants.

The best news is we’re well underway. Everyone should be proud of the pace of innovation over the last nine months — from long form content, to creator monetization, and tremendous advancements in brand safety protections. Our usage is at an all time high and we’ll continue to delight our entire community with new experiences in audio, video, messaging, payments, banking – creating a global marketplace for ideas, goods, services, and opportunities.

Please don’t take this moment for granted. You’re writing history, and there’s no limit to our transformation. And everyone, is invited to build X with us.

Elon and I will be working across every team and partner to bring X to the world. That includes keeping our entire community up to date, ensuring that we all have the information we need to move forward.

Now, let’s go make that next big impression on the world, together.

Linda'''

count_words(text)

In [ ]:
output = llm_chain.run(text)
print(count_words(output))
parse_text(output)

## Simple Chatbot

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [ ]:
instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input}"
system_prompt = "You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

In [ ]:

prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
llm_chain.predict(user_input="Hi, my name is Sam")

In [ ]:
llm_chain.predict(user_input="Can you tell me about yourself.")

In [ ]:
llm_chain.predict(user_input="Today is Friday. What number day of the week is that?")

In [ ]:
llm_chain.predict(user_input="what is the day today?")

In [ ]:
llm_chain.predict(user_input="What is my name?")

In [ ]:
llm_chain.predict(user_input="Can you tell me about the olympics")

In [ ]:
llm_chain.predict(user_input="What have we talked about in this Chat?")